In [1]:
# import the necessary libraries
import pandas as pd
import os

# set the path to the directory containing the Excel files
path = 'data'

# read in all the Excel files in the directory as a list of dataframes
dfs = [pd.read_excel(os.path.join(path, file)) for file in os.listdir(path) if file.endswith('.xlsx')]

# add a new column to each dataframe called 'pk' which should be the file name + index
for i, df in enumerate(dfs):
    df['pk'] = os.listdir(path)[i].split('.')[0] + '_' + df.index.astype(str)

# concatenate the list of dataframes into one big dataframe
master = pd.concat(dfs, ignore_index=True)

In [3]:
# # print out a list of all the column names
# print(master.columns)

Index(['Review Search', 'Search Token', 'Inputs First Name',
       'Inputs Middle Name', 'Inputs Last Name', 'Inputs SSN', 'Inputs DOB',
       'Inputs State', 'Inputs Product', 'Inputs Quoteback',
       'Inputs Include NULL DOBs', 'Age Post Filter',
       'Inputs Match Probability', 'Inputs Exact DOB Match Only',
       'Inputs Exact Name Match Only', 'Inputs Include Sex Offender Data',
       'Code', 'Code Message', 'Records Found', 'Category',
       'Source or Jurisdiction', 'Cleansed County', 'FIPS Code', 'State',
       'Full Name', 'First Name', 'Middle Name', 'Last Name', 'Suffix',
       'Full DOB', 'Birthstate', 'SSN', 'Sex', 'Race', 'Hair Color',
       'Eye Color', 'Weight', 'Height', 'Scars and Marks', 'Skin Tone',
       'Military Service', 'Age', 'Age at Death', 'Drivers License Number',
       'Death Verification Code', 'Case Number', 'County or Jurisdiction',
       'Photo Name', 'Status', 'Record Comments', 'Aliases', 'Addresses',
       'Offense Description', 'Arr

,Review Search,Search Token,Inputs First Name,Inputs Middle Name,Inputs Last Name,Inputs SSN,Inputs DOB,Inputs State,Inputs Product,Inputs Quoteback,...,Commitment State,Commitment Date,Release Date,FBI Number,Booking Number,Restitution,Comments,NextRescreen,Change/Update,pk
0,https://www.innovativedatasolutions.com/Index....,b79374a8-aae6-4a35-aaa7-8ca68a2ea76e,JOSHUA,GABRIEL,DILLON,NaN,05/17/1985,NaN,NCIS,30247857,...,NaN,NaN,05/22/2017,NaN,2995663,NaN,CUSTODY STATUS: RELEASED; RELEASE TYPE: SENTEN...,NaN,NaN,NCIS_1_0
1,https://www.innovativedatasolutions.com/Index....,3f2f7bac-813e-4bfc-8c2b-468a41c17ffd,CHRISTOPHER,ELISHA,GILBERT,NaN,06/06/1995,NaN,NCIS,30239997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NCIS_1_1
2,https://www.innovativedatasolutions.com/Index....,2dcd4ae8-8fb5-4df5-a9bc-d881c763e61d,SHA,NaN,MCKENZIE,NaN,08/27/1973,NaN,NCIS,30401493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NCIS_1_2
3,https://www.innovativedatasolutions.com/Index....,8bf4d525-e5f0-4cb3-a109-d8b8647f1da8,NATASHA,NOI,MCLEAN,NaN,05/24/1995,NaN,NCIS,30428173,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NCIS_1_3
4,https://www.innovativedatasolutions.com/Index....,18179086-8538-47fa-bdbf-be4bfc89c0c8,BRIANCA,SHANTE,CHILDS,NaN,01/04/1991,NaN,NCIS,30757797,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NCIS_1_4


In [4]:
# if the value of column 'code' in df master is 101 then ignore the row

In [5]:
# ____ DOB ____
# convert the 'Full DOB' column to a string
master['Full DOB'] = master['Full DOB'].astype(str)

# make a new column called 'cleaned_dob' and save the cleaned dob in that column
master['cleaned_dob'] = master['Full DOB']

# if len of 'Full DOB' is 4 then all we have is year and format it to '00/00/xxxx'
master['cleaned_dob'] = master['Full DOB'].apply(lambda dob: '00/00/' + dob if len(dob) == 4 else dob)

# convert the 'Full DOB' column to the same format the format is 'mmddyyyy'
master['cleaned_dob'] = master['Full DOB'].apply(lambda dob: dob[4:6] + '/' + dob[6:8] + '/' + dob[:4] if len(dob) == 8 else dob)

# define a function to check for matches
def check_match(row):
    cleaned_dob = row['cleaned_dob']
    inputs_dob = row['Inputs DOB']
    if cleaned_dob == 'nan':
        return 'no match'
    elif cleaned_dob == inputs_dob:
        return 'exact match'
    else:
        cleaned_dob_parts = cleaned_dob.split('/')
        inputs_dob_parts = inputs_dob.split('/')
        if len(cleaned_dob_parts) < 3 or len(inputs_dob_parts) < 3:
            return 'no match'
        else:
            match_types = {
                'year and month match': [2, 0],
                'year and day match': [2, 1],
                'month and day match': [0, 1],
                'year match': [2],
                'month match': [0],
                'day match': [1],
                'no match': []
            }
            for match_type, match_indices in match_types.items():
                if all(cleaned_dob_parts[idx] == inputs_dob_parts[idx] for idx in match_indices):
                    return match_type
            else:
                return 'no match'

# apply the function to each row of the dataframe
master['dob_match_finding'] = master.apply(check_match, axis=1)

# valuecounts of column 'dob_match_finding'
print(master['dob_match_finding'].value_counts())

year match              11116
exact match              8807
no match                 2270
month and day match      1136
year and month match      512
Name: dob_match_finding, dtype: int64


In [ ]:
# next steps with this is to do matching with the names to see if we have full or partial matches. 